In [0]:
threshold_amount = dbutils.widgets.get("threshold")
txns_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("dbfs:/Volumes/inceptez_catalog/inputdb/customerdata/txns") \
	.toDF("txnid","txndate","custid","amount","prodcategory","prodname","city","state","paymenttype").limit(1000)

In [0]:
# Calculate average sales amount
from pyspark.sql.functions import avg, round

avg_sales_df = txns_df.select(round(avg("amount"), 2).alias("avg_sales_amount"))
avg_sales_value = avg_sales_df.collect()[0]["avg_sales_amount"]

print(f"Average Sales Amount: ₹{avg_sales_value}")

In [0]:
# Store the average sales value as a task value
dbutils.jobs.taskValues.set("avg_sales_amount", avg_sales_value)
dbutils.jobs.taskValues.set("threshold_amount", threshold_amount)

print(" Task Value 'avg_sales_amount' set successfully.")

In [0]:
#write the data into the table
txns_df.write.format("delta").mode("overwrite").saveAsTable("inceptez_catalog.outputdb.tbltnxscurated")
print("Data written to table")